## Cacluate The amount of times each team has spent tied, leading, trailing, ect 

In [9]:
import pandas as pd

# path to single game file
# path = '../TEMP/single_game_scoring.csv'
# path = '../TEMP/ss_example_3.csv'

def calculate_game_states_dataframe_ot_tie_corrected_v2(file_path):
    data = pd.read_csv(file_path)

    def period_elapsed_to_seconds(period, time_str):
        minutes, seconds = map(int, time_str.split(':'))
        elapsed_seconds = minutes * 60 + seconds
        
        if '1st' in period:
            return elapsed_seconds
        elif '2nd' in period:
            return 20 * 60 + elapsed_seconds
        elif '3rd' in period:
            return 40 * 60 + elapsed_seconds
        else:  # Overtime
            return 60 * 60 + elapsed_seconds

    data['Total_Seconds'] = data.apply(lambda row: period_elapsed_to_seconds(row['Period'], row['Time']), axis=1)

    teams = list(set(data['Home_Team'].unique().tolist() + data['Away_Team'].unique().tolist()))
    team_states = {team: {'Tied': 0, 'Lead by 1': 0, 'Lead by 2': 0, 'Lead by 3+': 0,
                          'Down by 1': 0, 'Down by 2': 0, 'Down by 3+': 0} for team in teams}
    
    current_score = {teams[0]: 0, teams[1]: 0}
    last_timestamp = 0
    game_end = 3 * 20 * 60  # Default game end, will adjust if OT is detected
    overtime_played = False

    first_goal_time = data.sort_values(by='Total_Seconds').iloc[0]['Total_Seconds'] if not data.empty else 0
    team_states[teams[0]]['Tied'] = first_goal_time
    team_states[teams[1]]['Tied'] = first_goal_time
    last_timestamp = first_goal_time

    for index, row in data.sort_values(by='Total_Seconds').iterrows():
        current_score[row['Team']] += 1
        lead_diff = current_score[teams[0]] - current_score[teams[1]]
        time_elapsed = row['Total_Seconds'] - last_timestamp

        if 'Overtime' in row['Period']:
            overtime_played = True
            game_end = row['Total_Seconds']  # Adjust game end to when OT goal is scored

        if lead_diff == 0:
            for team in teams:
                team_states[team]['Tied'] += time_elapsed
        else:
            leading_team, trailing_team = (teams[0], teams[1]) if lead_diff > 0 else (teams[1], teams[0])
            lead_abs = abs(lead_diff)
            update_states = lambda lead: (f'Lead by {min(lead, 3)}+' if lead >= 3 else f'Lead by {lead}',
                                          f'Down by {min(lead, 3)}+' if lead >= 3 else f'Down by {lead}')
            state_lead, state_trail = update_states(lead_abs)
            team_states[leading_team][state_lead] += time_elapsed
            team_states[trailing_team][state_trail] += time_elapsed
        
        last_timestamp = row['Total_Seconds']

    # Determine if game ended in a tie without explicit OT goal and adjust game_end correctly
    lead_diff_final = current_score[teams[0]] - current_score[teams[1]]
    if lead_diff_final == 0 and not overtime_played:
        # Adjust for games that end in a tie without 'OT' period goals
        overtime_played = True
        game_end = 65 * 60  # Adjust game end to include 5 minutes of OT

    final_time_elapsed = game_end - last_timestamp
    if lead_diff_final == 0 and not data.empty:
        for team in teams:
            team_states[team]['Tied'] += final_time_elapsed

    team_states_df = pd.DataFrame.from_dict(team_states, orient='index').reset_index().rename(columns={'index': 'Team'})
    
    return team_states_df

# Re-generate the DataFrame with the corrected function for the new file
# game_states_df_ot_tie_corrected_v2 = calculate_game_states_dataframe_ot_tie_corrected_v2(path)
# game_states_df_ot_tie_corrected_v2


# Start Here

In [10]:
### Path to Target Database file
import os
import sqlite3
import pandas as pd

# path to database file using os module
# db_path = os.path.join('..', 'data', 'db', 'CHN_YTD_Stats.db') # Path to most recent cleaned DB File
db_path = os.path.join('..', 'data', 'db', '2020_Box_Scores.db') # Path to most recent cleaned DB File
conn = sqlite3.connect(db_path)

# verify the connection
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

In [11]:
## HOTFIX FOR OLD DB FILE - Add Home_Team and Away_Team to  scoring summary table in the db by spliting the Game_ID
## Home split '-' -1
## Away split '-' -2

# # example # Step 2: Create new columns for Home and Away Teams by parsing Game_ID
# df_game_details['Away_Team'] = df_game_details['Game_ID'].apply(lambda x: x.split('-')[3])
# df_game_details['Home_Team'] = df_game_details['Game_ID'].apply(lambda x: x.split('-')[4])



In [12]:
# season_file_path = '../TEMP/scoring_summary_old.csv'
# season_file_path = '../TEMP/Season_scoring.csv'

# extract scoring table from database
scoring_query = "SELECT * FROM scoring_summary"
scoring_df = pd.read_sql_query(scoring_query, conn)

## HOTFIX FOR OLD DB FILEs - Add Home_Team and Away_Team to  scoring summary table in the db by spliting the Game_ID
scoring_df['Away_Team'] = scoring_df['Game_ID'].apply(lambda x: x.split('-')[3])
scoring_df['Home_Team'] = scoring_df['Game_ID'].apply(lambda x: x.split('-')[4])


from tqdm import tqdm  # For progress tracking




# Function to process each game in the dataset and aggregate game states
def process_season_game_states(season_data):
    # season_data = pd.read_csv(file_path)
    unique_games = season_data['Game_ID'].unique()
    
    # Initialize an empty DataFrame for aggregating results
    season_aggregate_states = pd.DataFrame()
    
    for game_id in tqdm(unique_games, desc="Processing Games"):
        game_data = season_data[season_data['Game_ID'] == game_id]
        
        # Save individual game data to a temporary CSV to use with the existing function
        temp_game_path = '../TEMP/temp_game.csv'
        game_data.to_csv(temp_game_path, index=False)
        
        # Calculate game states for the current game
        game_states_df = calculate_game_states_dataframe_ot_tie_corrected_v2(temp_game_path)
        
        # Aggregate the results
        if season_aggregate_states.empty:
            season_aggregate_states = game_states_df
        else:
            season_aggregate_states = season_aggregate_states.merge(game_states_df, on='Team', how='outer')
            for state in ['Tied', 'Lead by 1', 'Lead by 2', 'Lead by 3+', 'Down by 1', 'Down by 2', 'Down by 3+']:
                season_aggregate_states[state] = season_aggregate_states[f'{state}_x'].fillna(0) + season_aggregate_states[f'{state}_y'].fillna(0)
                season_aggregate_states.drop([f'{state}_x', f'{state}_y'], axis=1, inplace=True)
    
    return season_aggregate_states

# Process the season dataset
season_aggregate_states_df = process_season_game_states(scoring_df)

season_aggregate_states_df

Processing Games: 100%|██████████| 262/262 [00:03<00:00, 66.06it/s]


,Team,Tied,Lead by 1,Lead by 2,Lead by 3+,Down by 1,Down by 2,Down by 3+
0,Notre Dame,19681.0,6114.0,4358.0,213.0,5633.0,4204.0,615.0
1,Wisconsin,12710.0,3423.0,6540.0,3326.0,3626.0,3533.0,3551.0
2,Adrian,1288.0,0.0,0.0,0.0,0.0,3659.0,4893.0
3,Bowling Green,16414.0,5528.0,9001.0,8897.0,4201.0,2542.0,0.0
4,Michigan,15622.0,7313.0,3748.0,8629.0,668.0,3970.0,1820.0
5,Arizona State,16966.0,5106.0,4693.0,2228.0,9901.0,6923.0,6666.0
6,Holy Cross,9634.0,5600.0,1388.0,2840.0,1277.0,1562.0,2798.0
7,Long Island,9739.0,1405.0,500.0,0.0,4145.0,3172.0,4116.0
8,Michigan State,21163.0,4315.0,3090.0,1645.0,4297.0,5858.0,3794.0
9,Penn State,11089.0,6240.0,2775.0,740.0,7267.0,8333.0,6131.0


In [13]:
# Sort by Down by 3+ time to find the teams that were trailing by 3+ goals the most
# season_aggregate_stats_df.sort_values(by='Down by 3+', ascending=True).head(10)
season_aggregate_stats_df = season_aggregate_states_df

In [14]:
### Calculate the percentages for the different game states
# Calculate the total game time for each team
season_aggregate_stats_df['Total_Time'] = season_aggregate_stats_df['Tied'] + season_aggregate_stats_df['Lead by 1'] + season_aggregate_stats_df['Lead by 2'] + season_aggregate_stats_df['Lead by 3+'] + season_aggregate_stats_df['Down by 1'] + season_aggregate_stats_df['Down by 2'] + season_aggregate_stats_df['Down by 3+']

# create column of all leads and all deficits
season_aggregate_stats_df['All_Lead'] = season_aggregate_stats_df['Lead by 1'] + season_aggregate_stats_df['Lead by 2'] + season_aggregate_stats_df['Lead by 3+']
season_aggregate_stats_df['All_Deficit'] = season_aggregate_stats_df['Down by 1'] + season_aggregate_stats_df['Down by 2'] + season_aggregate_stats_df['Down by 3+']

# Calculate the percentages for each game state
for state in ['Tied', 'Lead by 1', 'Lead by 2', 'Lead by 3+', 'Down by 1', 'Down by 2', 'Down by 3+', 'All_Lead', 'All_Deficit']:
    season_aggregate_stats_df[f'PCT_{state}'] = season_aggregate_stats_df[state] / season_aggregate_stats_df['Total_Time']

# Display the DataFrame with the calculated percentages
# season_aggregate_stats_df

In [15]:
# Convert all time columns from seconds to minutes
for state in ['Tied', 'Lead by 1', 'Lead by 2', 'Lead by 3+', 'Down by 1', 'Down by 2', 'Down by 3+', 'All_Lead', 'All_Deficit', 'Total_Time']:
    season_aggregate_stats_df[state] = season_aggregate_stats_df[state] / 60

# Display the DataFrame with the times converted to minutes
season_aggregate_stats_df


,Team,Tied,Lead by 1,Lead by 2,Lead by 3+,Down by 1,Down by 2,Down by 3+,Total_Time,All_Lead,All_Deficit,PCT_Tied,PCT_Lead by 1,PCT_Lead by 2,PCT_Lead by 3+,PCT_Down by 1,PCT_Down by 2,PCT_Down by 3+,PCT_All_Lead,PCT_All_Deficit
0,Notre Dame,328.016667,101.900000,72.633333,3.550000,93.883333,70.066667,10.250000,680.300000,178.083333,174.200000,0.482165,0.149787,0.106767,0.005218,0.138003,0.102994,0.015067,0.261772,0.256064
1,Wisconsin,211.833333,57.050000,109.000000,55.433333,60.433333,58.883333,59.183333,611.816667,221.483333,178.500000,0.346237,0.093247,0.178158,0.090604,0.098777,0.096243,0.096734,0.362009,0.291754
2,Adrian,21.466667,0.000000,0.000000,0.000000,0.000000,60.983333,81.550000,164.000000,0.000000,142.533333,0.130894,0.000000,0.000000,0.000000,0.000000,0.371850,0.497256,0.000000,0.869106
3,Bowling Green,273.566667,92.133333,150.016667,148.283333,70.016667,42.366667,0.000000,776.383333,390.433333,112.383333,0.352360,0.118670,0.193225,0.190992,0.090183,0.054569,0.000000,0.502887,0.144752
4,Michigan,260.366667,121.883333,62.466667,143.816667,11.133333,66.166667,30.333333,696.166667,328.166667,107.633333,0.374000,0.175078,0.089729,0.206584,0.015992,0.095044,0.043572,0.471391,0.154609
5,Arizona State,282.766667,85.100000,78.216667,37.133333,165.016667,115.383333,111.100000,874.716667,200.450000,391.500000,0.323267,0.097289,0.089419,0.042452,0.188652,0.131909,0.127013,0.229160,0.447573
6,Holy Cross,160.566667,93.333333,23.133333,47.333333,21.283333,26.033333,46.633333,418.316667,163.800000,93.950000,0.383840,0.223116,0.055301,0.113152,0.050879,0.062234,0.111479,0.391569,0.224591
7,Long Island,162.316667,23.416667,8.333333,0.000000,69.083333,52.866667,68.600000,384.616667,31.750000,190.550000,0.422022,0.060883,0.021667,0.000000,0.179616,0.137453,0.178359,0.082550,0.495428
8,Michigan State,352.716667,71.916667,51.500000,27.416667,71.616667,97.633333,63.233333,736.033333,150.833333,232.483333,0.479213,0.097708,0.069970,0.037249,0.097301,0.132648,0.085911,0.204927,0.315860
9,Penn State,184.816667,104.000000,46.250000,12.333333,121.116667,138.883333,102.183333,709.583333,162.583333,362.183333,0.260458,0.146565,0.065179,0.017381,0.170687,0.195725,0.144005,0.229125,0.510417


In [16]:
## Output the results to a CSV file
# output_file_path = '../TEMP/lead_Trail_season_aggregate_states.csv' # Original Path - power BI report is tied to tis file location
output_file_path = '../data/2020_Lead_Trail_season_aggregate_stats.csv'
season_aggregate_stats_df.to_csv(output_file_path, index=False)